See readme.md for ideal fields and descriptions

In [1]:
# Can do installs here when conde env is activated
# ! pip install pandas pyarrow
# ! pip install polars
# ! pip install maturin
# ! pip install cryo
# ! pip install web3

In [2]:
import os
import glob
from datetime import datetime, timedelta, timezone
import time
import pyarrow.parquet as pq
import pandas as pd
import cryo
import sys
sys.path.append("../../helper_functions")
import web3py_utils as w3py
import os_utils as osu
sys.path.pop()
import polars as pl
# test adding cryo_cli python

/Users/michaelsilberling/opt/anaconda3/envs/cryoenv/lib/python3.10/site-packages/eth_utils/network.py:61: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  networks = initialize_network_objects()
/Users/michaelsilberling/opt/anaconda3/envs/cryoenv/lib/python3.10/site-packages/eth_utils/network.py:61: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  networks = initialize_network_objects()


In [3]:
output_directory = 'cryo_outputs/'
# clear out
osu.clear_folder(output_directory)

The directory does not exist.


In [4]:

config_chain = {
        'rpc_url': 'https://rpc.redstonechain.com',
        'chain_name': 'redstone',
        'block_time_sec': 2,
        'block_time_buffer': 0
        }
# Lyra
# 'rpc_url': 'https://rpc.lyra.finance/'
# 'chain_name': 'lyra'
# 'block_time_sec': 2
# 'block_time_buffer': 0
# DEGEN
# 'rpc_url': 'https://rpc.degen.tips',
# 'chain_name': 'degen',
# 'block_time_sec': 0.383, #Note: Arb Stack not deterministic
# 'block_time_buffer': 0.25

###
whole_day_only = False#True
trailing_days = 0.25
dry_run = False
fields = ['blocks', 'txs']
requests_per_second_max = 500 # -1 means ignore


In [5]:
#Intermediate Calc
rpc_url = config_chain['rpc_url']
chain_name = config_chain['chain_name']
block_time_sec = config_chain['block_time_sec']
block_time_buffer = config_chain['block_time_buffer']

blocks_per_day = (60*60*24) / block_time_sec
blocks_per_day_lo = (60*60*24) / (block_time_sec+block_time_buffer)
if block_time_sec-block_time_buffer > 0:
        blocks_per_day_hi = (60*60*24) / (block_time_sec-block_time_buffer)
else:
        blocks_per_day_hi = (60*60*24) / 0.01

print(blocks_per_day)

43200.0


In [6]:
# Init timestamps

# Get the current time in UTC
current_time_utc = datetime.utcnow()
print(current_time_utc)

# If only whole days then shift the ending time to the start of the day
if whole_day_only:
        current_date_utc = datetime.combine(datetime.utcnow().date(), datetime.min.time())
        time_difference = current_time_utc - current_date_utc
        difference_days_fraction = time_difference.total_seconds() / (24 * 3600)  # There are 86400 seconds in a day
else: 
        current_date_utc = current_time_utc
        difference_days_fraction = 0
print('day fraction :' + str(difference_days_fraction))

print(current_date_utc)
starting_date_utc = current_date_utc - timedelta(days=trailing_days)

current_block = w3py.getLatestBlockNumber(rpc_url)

ending_block = int( current_block - (difference_days_fraction * blocks_per_day_lo) )
starting_block = int( ending_block - (trailing_days * blocks_per_day_hi) )

print('current: ' + str(int(current_block)))
print('end: ' + str(int(ending_block)))
print('start: ' + str(int(starting_block)))

2024-05-02 01:17:28.934249
day fraction :0
2024-05-02 01:17:28.934249
current: 1213779
end: 1213779
start: 1202979


In [7]:
start_timestamp = int(starting_date_utc.timestamp())
end_timestamp = int(current_date_utc.timestamp())

start_time = time.time()

In [8]:
# Fetch and save blocks data in JSON
data = cryo.freeze(
    ['txs', 'blocks'],
    blocks=[str(starting_block) + ":" + str(ending_block)],
    rpc=rpc_url,
    output_dir= output_directory,
    file_format='parquet',
    label=chain_name,
    hex=True,
    dry=dry_run,
    requests_per_second=requests_per_second_max
)

cryo parameters
───────────────
- version: 152a635
- data: 
    - datatypes: blocks, transactions
    - blocks: n=10,800 min=1,202,979 max=1,213,778 align=no reorg_buffer=0
- source: 
    - network: network_690
    - rpc url: https://rpc.redstonechain.com
    - max requests per second: 500
    - max concurrent requests: 500
    - max concurrent chunks: 4
- output: 
    - chunk size: 1,000
    - chunks to collect: 11 / 11
    - output format: parquet
    - output dir: /Users/michaelsilberling/Documents Local/GitHub/op-analytics/op_chains_tracking/lightweight_chain_tracker/cryo_outputs
    - report file: $OUTPUT_DIR/.cryo/reports/2024-05-01_21-17-30.506071.json


schema for blocks
─────────────────
- block_number: uint32
- block_hash: hex
- timestamp: uint32
- author: hex
- gas_used: uint64
- extra_data: hex
- base_fee_per_gas: uint64
- chain_id: uint64

sorting blocks by: block_number

other available columns: parent_hash, state_root, transactions_root, receipts_root, logs_bloom, total_

In [9]:
# Calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.4f} seconds")

Elapsed time: 1695.8899 seconds


In [10]:
# Move this to a cryo_utils eventually

def load_parquet_files(chain_name, data_type_name, data_directory):
    pattern = f"*{data_type_name}__{chain_name}*.parquet"
    file_paths = glob.glob(os.path.join(data_directory, pattern))
    
    if file_paths:
        df = pl.scan_parquet(file_paths)
        # Further processing can go here, for example:
        # df = df.filter(pl.col("some_column") > 0)
        return df.collect()  # Collecting after all transformations
    else:
        print("No files found matching the pattern.")
        return None

In [11]:
# # Read parquet files
txs = load_parquet_files(chain_name, 'transactions', output_directory)
blocks = load_parquet_files(chain_name, 'blocks', output_directory)

# Rename the 'gas_used' column to 'block_gas_used' in the 'blocks' DataFrame
blocks = blocks.rename({"gas_used": "block_gas_used"})

# Perform the join on 'block_number' and 'chain_id'
joined_df = blocks.join(
    txs,
    on=["block_number", "chain_id"],
    how="inner"  # You can specify the type of join you want (inner, outer, left, right)
)

# Convert Unix timestamp to datetime and create a new column 'timestamp_dt'
joined_df = joined_df.with_columns(
    pl.from_epoch("timestamp", time_unit="s").alias("timestamp_dt")
)

# Truncate the 'timestamp_dt' column to the day and create a new column 'timestamp_date'
joined_df = joined_df.with_columns(
    pl.col("timestamp_dt").dt.truncate("1d").alias("timestamp_date")
)

In [12]:
# print(blocks.schema)
# print(txs.schema)
print(joined_df.schema)
print(type(joined_df))

#test output
joined_pd = joined_df.to_pandas()
# print(joined_pd.tail(5))

print('num blocks: ' + str(joined_pd['block_number'].nunique()))

OrderedDict([('block_hash', Utf8), ('author', Utf8), ('block_number', UInt32), ('block_gas_used', UInt64), ('extra_data', Utf8), ('timestamp', UInt32), ('base_fee_per_gas', UInt64), ('chain_id', UInt64), ('transaction_index', UInt64), ('transaction_hash', Utf8), ('nonce', UInt64), ('from_address', Utf8), ('to_address', Utf8), ('value_binary', Utf8), ('value_string', Utf8), ('value_f64', Float64), ('input', Utf8), ('gas_limit', UInt64), ('gas_used', UInt64), ('gas_price', UInt64), ('transaction_type', UInt32), ('max_priority_fee_per_gas', UInt64), ('max_fee_per_gas', UInt64), ('success', Boolean), ('n_input_bytes', UInt32), ('n_input_zero_bytes', UInt32), ('n_input_nonzero_bytes', UInt32), ('timestamp_dt', Datetime(time_unit='us', time_zone=None)), ('timestamp_date', Datetime(time_unit='us', time_zone=None))])
<class 'polars.dataframe.frame.DataFrame'>
num blocks: 10800


In [13]:
# Assuming you have a DataFrame named 'joined_df' with the required columns

result_df = joined_df.group_by([pl.col("timestamp_date"), pl.col("chain_id")]).agg(
    num_blocks=pl.col("block_number").n_unique(),
    min_block_number=pl.col("block_number").min(),
    max_block_number=pl.col("block_number").max(),
    min_block_time=pl.col("timestamp").min(),
    max_block_time=pl.col("timestamp").max(),

    num_user_transactions=
        pl.when(pl.col("gas_price") > 0).then(pl.col("transaction_hash")).count(),
    num_success_user_transactions=
        pl.when((pl.col("gas_price") > 0) & pl.col("success")).then(pl.col("transaction_hash")).count(),
    num_senders=pl.col("from_address").filter(pl.col("gas_price") > 0).n_unique(),

    total_gas_used=pl.col("gas_used").sum(),
    user_gas_used=pl.col("gas_used").filter(pl.col("gas_price") > 0).sum(),
    total_gas_used_per_block = pl.col("gas_used").sum() / pl.col("block_number").n_unique(),
    user_gas_used_per_block = pl.col("gas_used").filter(pl.col("gas_price") > 0).sum() / pl.col("block_number").n_unique(),
    
    l2_fees_base_fees_eth=(pl.col("base_fee_per_gas") * pl.col("gas_used")).sum() / 1e18,
    l2_fees_priority_fees_eth=pl.when(pl.col("gas_price") > 0).then((pl.col("gas_price") - pl.col("base_fee_per_gas")) * pl.col("gas_used")).sum() / 1e18,
    l2_fees_total_fees_eth=(pl.col("gas_price") * pl.col("gas_used")).sum() / 1e18,

    avg_l2_base_fee_gwei=(pl.col("base_fee_per_gas") * pl.col("gas_used")).sum()
                            / pl.col("gas_used").sum() /1e9,
    avg_l2_priority_fee_gwei=pl.when(pl.col("gas_price") > 0).then((pl.col("gas_price") - pl.col("base_fee_per_gas")) * pl.col("gas_used")).sum()
                            / pl.col("gas_used").sum() /1e9,
)
result_df = result_df.to_pandas()

In [14]:
# Filter
result_df = result_df[ (result_df['min_block_time']>= start_timestamp ) & (result_df['min_block_time']<= end_timestamp ) ]
#seems like 1 block before gets pulled. yolo.

In [15]:
result_df['min_block_time_dt'] = pd.to_datetime(result_df['min_block_time'], unit='s')
result_df['max_block_time_dt'] = pd.to_datetime(result_df['max_block_time'], unit='s')
display(result_df.sort_values(by='timestamp_date',ascending=False))

,timestamp_date,chain_id,num_blocks,min_block_number,max_block_number,min_block_time,max_block_time,num_user_transactions,num_success_user_transactions,num_senders,total_gas_used,user_gas_used,total_gas_used_per_block,user_gas_used_per_block,l2_fees_base_fees_eth,l2_fees_priority_fees_eth,l2_fees_total_fees_eth,min_block_time_dt,max_block_time_dt
1,2024-05-02,690,2324,1211455,1213778,1714608001,1714612647,8037,6864,810,25738923377,25620162573,1.107527e+07,1.102417e+07,6.434731e-07,0.086203,0.086204,2024-05-02 00:00:01,2024-05-02 01:17:27
